In [3]:
import wget
import os
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [5]:
PATH = r'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

In [ ]:
page_link = "https://www.domain.com.au/auction-results/melbourne/"

for num_1 in range(3,4):
    for num_2 in range(1,3):
        driver.get(page_link)
        try:
            print('/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[' + str(num_1) + ']/article[' + str(num_2) + ']/div/a[1]')
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[' + str(num_1) + ']/article[' + str(num_2) + ']/div/a[1]')))
            element = driver.find_element_by_xpath('/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[' + str(num_1) + ']/article[' + str(num_2) + ']/div/a[1]')
            
            element.click()
        except:
            break
        try:
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')))
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')
            print(element.text)
        except:
            pass

        try:
            current_url = driver.current_url 
            for num in range(1,100):
                if num == 1:
                    driver.get(current_url)
                else:
                    try:
                        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img')))
                        driver.get(current_url + "?page=" + str(num))
                        
                        for num in range(1,26):
                            try:
                                WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img')))
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(num) + ']/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img')
                                element.click()

                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[1]/div[1]/div')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[1]/div/div/div/p')
                                print(element.text)
                                element = driver.find_element_by_xpath('//*[@id="__domain_group/APP_ROOT"]/div/div/div[5]/div/div/div[5]/div/div/p')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[4]/span')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[1]/span')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[2]/span')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[3]/span')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/span[2]')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/aside/div[1]/div/div[1]/div/div/div/a[2]')
                                print(element.text)
                                /html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span/span
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div')
                                print(element.text)
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[9]/div/div/section/div[5]/button[1]')
                                element.click()
                                element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[9]/div/div/section')
                                print(element.text)
                            except:
                                pass
                    except:
                        break
        except:
            pass

/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[3]/article[1]/div/a[1]
21564 Properties sold in Abbotsford, VIC, 3067


In [14]:
page_link = "https://www.domain.com.au/auction-results/melbourne/"
driver.get(page_link)
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[3]/article[1]/div/a[1]')))
    element = driver.find_element_by_xpath('/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[3]/article[1]/div/a[1]')
    element.click()
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')))
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')
    print(element.text)
    WebDriverWait(driver, 2)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img')
    element.click()
    WebDriverWait(driver, 2)
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[2]/button')))
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[2]/button')
    element.click()
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div')))
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[1]/div[1]/div')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[1]/div/div/div/p')
    print(element.text)
    element = driver.find_element_by_xpath('//*[@id="__domain_group/APP_ROOT"]/div/div/div[5]/div/div/div[5]/div/div/p')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[4]/span')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[1]/span')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[2]/span')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[3]/span')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/span[2]')
    print(element.text)
    element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/aside/div[1]/div/div[1]/div/div/div/a[2]')
    print(element.text)

1605 Properties sold in Aireys Inlet, VIC, 3231


In [ ]:
page_link = "https://www.domain.com.au/auction-results/melbourne/"
driver.get(page_link)
WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[3]/article[1]/div/a[1]')))
element = driver.find_element_by_xpath('/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[3]/article[1]/div/a[1]')
element.click()
WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')))
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')
print(element.text)
WebDriverWait(driver, 2)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img
element.click()
WebDriverWait(driver, 2)
WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[2]/button')))
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[2]/button')
element.click()
WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div')))
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[1]/div[1]/div')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[1]/div/div/div/p')
print(element.text)
element = driver.find_element_by_xpath('//*[@id="__domain_group/APP_ROOT"]/div/div/div[5]/div/div/div[5]/div/div/p')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[4]/span')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[1]/span')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[2]/span')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span[3]/span')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/span[2]')
print(element.text)
element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/aside/div[1]/div/div[1]/div/div/div/a[2]')
print(element.text)
/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div/span/span